<a href="https://colab.research.google.com/github/shamitb/BerkeleyX-CS100.1x-Big-Data-with-Apache-Spark/blob/master/LATAM_Manufac_COVID_Impac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Installations

!pip install pytrends
!pip install yfinance
!pip install forex-python
!pip install datetime
!pip install pytrends --upgrade 


In [0]:
#@title Imports
import pandas as pd
import matplotlib as plt
from pytrends.request import TrendReq
import yfinance as yf
from google.colab import files
from datetime import timedelta, date, datetime

Search trends:
Compare y(top 3 loc) vs y competition
Related queries for y company (top x queries)
Understanding top 3 - 5 loc by sub-region/city
~Localised competition view~



In [0]:
#@title Variable and Function Initialisation
pytrends = TrendReq(hl='en-UK', tz=60)
cpy = ["Whirlpool"] # company
cpts = ["Electrolux"] # competitors
r = ['AR', 'BR', 'MX'] # countries to search
res = 'CITY' # resolution for region search 'CITY' returns city level data 'COUNTRY' returns country level data 'DMA' returns Metro level data 'REGION' returns Region level data
kw_list = cpy + cpts # list of company and competitors
tf = '2020-02-03 2020-04-20' # timeframe for search 


def to_csv(df, filename):
  '''
  @params: df -> dataframe, filename -> name of file to be created
  converts the inputed dataframe into a csv file and downloads the file
  '''
  df.to_csv(filename)
  files.download(filename)



def date_range(start, end):
  '''
  @params: start -> start date, end -> end date
  yields a list of dates from start to end
  yield is like return however it returns a generator
  a generator is like an iterable (list) but will forget the value after it has 
  been read
  '''
  for n in range(int( (end - start).days )):
    yield start + timedelta(n)


def region_interest(start,end, df, comp_x):
  '''
  @params: start -> start date for search, end -> end date for search
  @params: df -> dataframe that values will be appended to, comp_x -> dataframe of related topics for each day in time frame
  returns a new dataframe with the interest of a particular topic ploted over 
  all regions of a country for each day between start and end
  '''

  comp_x_2 = comp_x.loc[start:end]

  for index, _ in comp_x_2.iterrows(): 
    range_date = datetime.strptime(comp_x.iloc[index]['date'],"%Y-%m-%d") - dd
    st = {'title': [comp_x.iloc[index]['topic_title']], 
          'date':range_date.strftime("%Y-%m-%d")+' '+comp_x.iloc[index]['date']}
    print(index, end='')
    
    pytrends.build_payload(st['title'],timeframe=st['date'],geo=r[0])

    region_trends = pytrends.interest_by_region(res)

    df[st['title']+str(index)] = region_trends[st['title']]
    df['date_'+str(index)] = comp_x.iloc[index]['date']
    
    print(st['title']+' '+st['date'],end=' ')
    print('f')
  
  return df


#def get_city_trends(reg,comp):
 # pytrends.build_payload(comp,cat=0,timeframe=tf,geo=reg,gprop='')
  #return pytrends.interest_by_region(res)

#city_trends = {c:get_city_trends(r[0],[c]) for c in kw_list}
#print(city_trends)


#city_trends.plot()
#comp_x_top = pd.DataFrame(pytrends.related_queries()[cpy[0]]['top'])
#comp_x_top.head()

#comp_x_rising = pd.DataFrame(pytrends.related_queries()[cpy[0]]['rising'])
#comp_x_rising.head()



In [0]:
'''
for each country in r, creates a dataframe for the search interest of each 
company in kw_list and downloads that dataframe in a csv file
'''

for cont in r:
  pytrends.build_payload(kw_list, cat=0, timeframe=tf, geo=cont, gprop='')
  trends = pytrends.interest_over_time()
  filename = cont + "_search_trends_for_country.csv"
  to_csv(trends, filename)

In [0]:
start = date(2020, 2, 3)
end = date(2020, 4, 21)
comp_x = pd.DataFrame()

'''
for each date between start and end, returns a dataframe containing the related 
topics for the main company for that day. The dataframe is then appended to a 
different dataframe.
'''

for curr_date in date_range(start, end):
  dd = timedelta(days=4)
  frame = curr_date - dd
  frame_str = frame.strftime("%Y-%m-%d") + " " + curr_date.strftime("%Y-%m-%d")
  '''
  The api doesn't allow for only one date to be queried so to get around this the
  search is started 4 days back
  '''
  #print(frame_str)

  pytrends.build_payload(cpy, cat=0, timeframe=frame_str, geo=r[0], gprop='')
  related_topics = pytrends.related_topics()
  tmp = related_topics[cpy[0]]['rising']
  if not tmp.empty:
    comp_x_rt = tmp.drop(['link','topic_mid'], axis=1)
    comp_x_rt['date']=curr_date.strftime("%Y-%m-%d")
    comp_x = comp_x.append(comp_x_rt)

comp_x = comp_x.reset_index()
comp_x.head(20)
filename = r[0] + "_related_topics_per_day.csv"
'''
The main dataframe is then saved as a csv file
'''
to_csv(comp_x, filename)

In [0]:
#@title Exchange rate
from forex_python.converter import CurrencyRates
from time import sleep

'''
Some testing on whether it is possible to use a python library to output the 
exchange rate for different currencies. The library is quite slow and only 
allows queriers every x seconds (I haven't worked out the exact time)
'''

c = CurrencyRates()
brl = c.get_rate('BRL','USD')
'''
sleep(10)
ars = c.get_rate('USD','ARS')
sleep(10)
mxn = c.get_rate('USD','MXN')
'''
data = {'brl': brl}
#, 'ars': ars, 'mxn': mxn
current_rate = pd.Series(data)

print(current_rate)


Twitter:
Word cloud -> what are people talking about
